# Initilize

In [ ]:
# Libraries
from pathlib import Path
from pprint import pprint
import json

In [ ]:
# Variables
isci = 'TEST24PH'
current_directory = 'reference\\'
edl_path = f'{current_directory}{isci}.edl'
drop_frame = None
edit_hour_mark = ""  # variable set in detect_dropframe
dissolve_offset = ""  # variable set in detect_dropframe function.  this compensates for the fade in Vantage.  Vantage treats a fade where first frame is 0% opacity and the last frame is 100% opacity.  Avid treats the first & last frame as showing some of the video.  This cheats that look so first or last frame isn't blank.
framerate = 23.976  # Plugged into the cml_edit_code.
video_edits = []
audio_edits = []
media_files = []

In [ ]:
# Avid CML Code Snippets
cml_header_code = """<?xml version="1.0" encoding="utf-8"?>
<Composition xmlns="Telestream.Soa.Facility.Playlist">
	<Source identifier="999999">		<File location="{$$Workflow_SupportingFilesPath}\{$$Workflow_Master}" /> 
		<Subtitle />
	</Source>
"""

cml_media_code = """
<Source identifier="{source_number}">
		<File location="{{$$Workflow_SupportingFilesPath}}\{media_file}" />
	</Source>
"""

cml_slate_code = """	
<Sequence layer="1">
	<Segment>
		<Image layer="0" location="C:\\Users\\Administrator\\Desktop\\AutoConform\\AutoSlate\\PWSlate.png" duration="{$$SlateLengthTimecode}" layout="center" fill="loop" />		
		
			<Title align="head" adjust="edge" fill="none" layer="1" duration="{$$SlateLengthTimecode}"  font="Trebuchet MS" size="42pt" weight="bold" foreground-color="FF1b8fa2" background-color="transparent" wrap="true" horizontal-align="left" vertical-align="top" overflow="resize"> 
				{$$Workorder_Title} 
				<Area left="715px" right="1610px" bottom="970px" top="350px"  />
			</Title>
		
			<Title align="head" adjust="edge" fill="none" layer="1" duration="{$$SlateLengthTimecode}"  font="Trebuchet MS" size="42pt" weight="normal" foreground-color="FF525255" background-color="transparent" wrap="true" horizontal-align="left" overflow="resize" layout="stretch"> 
				{$$Workorder_Agency} 
				<Area left="715px" right="1610px" top="455px"  />
			</Title>
		
			<Title align="head" adjust="edge" fill="none" layer="1" duration="{$$SlateLengthTimecode}"  font="Trebuchet MS" size="42pt" weight="normal" foreground-color="FF525255" background-color="transparent" wrap="true" horizontal-align="left" overflow="resize" layout="stretch"> 
				{$$Workorder_Client} 
				<Area left="715px" right="1610px" top="510px"  />
			</Title>
		
			<Title align="head" adjust="edge" fill="none" layer="1" duration="{$$SlateLengthTimecode}"  font="Trebuchet MS" size="42pt" weight="normal" foreground-color="FF525255" background-color="transparent" wrap="true" horizontal-align="left" overflow="resize" layout="stretch"> 
				{$$Workorder_Date} 
				<Area left="715px" right="1610px" top="565px"  />
				</Title>
		
			<Title align="head" adjust="edge" fill="none" layer="1" duration="{$$SlateLengthTimecode}"  font="Trebuchet MS" size="42pt" weight="normal" foreground-color="FF525255" background-color="transparent" wrap="true" horizontal-align="left" overflow="resize" layout="stretch"> 
				{$$Workorder_TRT}
				<Area left="715px" right="1610px" top="620px"  />
			</Title>
		
			<Title align="head" adjust="edge" fill="none" layer="1" duration="{$$SlateLengthTimecode}"  font="Trebuchet MS" size="42pt" weight="normal" foreground-color="FF525255" background-color="transparent" wrap="true" horizontal-align="left" overflow="resize" layout="stretch"> 
				{$$Workorder_SlatedISCI} 
				<Area left="715px" right="1610px" top="675px"  />
			</Title>
		
			<Title align="head" adjust="edge" fill="none" layer="1" duration="{$$SlateLengthTimecode}"  font="Trebuchet MS" size="42pt" weight="normal" foreground-color="FF525255" background-color="transparent" wrap="true" horizontal-align="left" overflow="resize" layout="stretch"> 
				{$$Workflow_Slate_TFN} 
				<Area left="715px" right="1610px" top="730px"  />
			</Title>
		
			<Title align="head" adjust="edge" fill="none" layer="1" duration="{$$SlateLengthTimecode}"  font="Trebuchet MS" size="42pt" weight="normal" foreground-color="FF525255" background-color="transparent" wrap="true" horizontal-align="left" overflow="resize" layout="stretch"> 
				{$$Workflow_Slate_URL} 
				<Area left="715px" right="1610px" top="785px"  />
			</Title>
					
			<Title align="head" adjust="edge" fill="none" layer="1" duration="{$$SlateLengthTimecode}"  font="Trebuchet MS" size="42pt" weight="normal" foreground-color="FF525255" background-color="transparent" wrap="true" horizontal-align="left" overflow="resize" layout="stretch"> 
				{$$Workflow_Slate_Promo} 
				<Area left="715px"  right="1610px" top="840px" />
			</Title>			

			<Title align="head" adjust="edge" fill="none" layer="1" duration="{$$SlateLengthTimecode}"  font="Trebuchet MS" size="42pt" weight="bold" foreground-color="FF1b8fa2" background-color="transparent" wrap="true" horizontal-align="right" vertical-align="top" overflow="resize"> 
				Title:
				<Area left="310px" right="685px" bottom="970px" top="350px"  />
			</Title>	
		
			<Title align="head" adjust="edge" fill="none" layer="1" duration="{$$SlateLengthTimecode}"  font="Trebuchet MS" size="42pt" weight="bold" foreground-color="FF1b8fa2" background-color="transparent" wrap="true" horizontal-align="right" vertical-align="top" overflow="resize"> 
				Agency:
				<Area left="310px" right="685px" bottom="970px" top="455px"  />
			</Title>				


			<Title align="head" adjust="edge" fill="none" layer="1" duration="{$$SlateLengthTimecode}"  font="Trebuchet MS" size="42pt" weight="bold" foreground-color="FF1b8fa2" background-color="transparent" wrap="true" horizontal-align="right" vertical-align="top" overflow="resize"> 
				Client:
				<Area left="310px" right="685px" bottom="970px" top="510px"  />
			</Title>	
		
			<Title align="head" adjust="edge" fill="none" layer="1" duration="{$$SlateLengthTimecode}"  font="Trebuchet MS" size="42pt" weight="bold" foreground-color="FF1b8fa2" background-color="transparent" wrap="true" horizontal-align="right" vertical-align="top" overflow="resize"> 
				Date:
				<Area left="310px" right="685px" bottom="970px" top="565px"  />
			</Title>	
		
			<Title align="head" adjust="edge" fill="none" layer="1" duration="{$$SlateLengthTimecode}"  font="Trebuchet MS" size="42pt" weight="bold" foreground-color="FF1b8fa2" background-color="transparent" wrap="true" horizontal-align="right" vertical-align="top" overflow="resize"> 
				Length:
				<Area left="310px" right="685px" bottom="970px" top="620px"  />
			</Title>	
		
			<Title align="head" adjust="edge" fill="none" layer="1" duration="{$$SlateLengthTimecode}"  font="Trebuchet MS" size="42pt" weight="bold" foreground-color="FF1b8fa2" background-color="transparent" wrap="true" horizontal-align="right" vertical-align="top" overflow="resize"> 
				ISCI:
				<Area left="310px" right="685px" bottom="970px" top="675px"  />
			</Title>	
		
			<Title align="head" adjust="edge" fill="none" layer="1" duration="{$$SlateLengthTimecode}"  font="Trebuchet MS" size="42pt" weight="bold" foreground-color="FF1b8fa2" background-color="transparent" wrap="true" horizontal-align="right" vertical-align="top" overflow="resize"> 
				#:
				<Area left="310px" right="685px" bottom="970px" top="730px"  />
			</Title>	

			<Title align="head" adjust="edge" fill="none" layer="1" duration="{$$SlateLengthTimecode}"  font="Trebuchet MS" size="42pt" weight="bold" foreground-color="FF1b8fa2" background-color="transparent" wrap="true" horizontal-align="right" vertical-align="top" overflow="resize"> 
				URL:
				<Area left="310px" right="685px" bottom="970px" top="785px"  />
			</Title>	

			<Title align="head" adjust="edge" fill="none" layer="1" duration="{$$SlateLengthTimecode}"  font="Trebuchet MS" size="42pt" weight="bold" foreground-color="FF1b8fa2" background-color="transparent" wrap="true" horizontal-align="right" vertical-align="top" overflow="resize"> 
				{$$Workorder_PromoTitle}
				<Area left="310px" right="685px" bottom="970px" top="840px"  />
			</Title>
	</Segment>

	<Segment>
		<Canvas align="head" adjust="body" duration="{$$BlackHeadTimecode}" Background="black" layer="0" />                           
	</Segment>

	<Segment>
		<Video source="999999" />			
			"""

cml_edit_code = """
			<{media_type} source="{source_number}" align="head" adjust="edge" offset="{{ {edit_in}-{dissolve_offset}-{edit_hour_mark} }}" filter="mute" >
				<Head>
					<Fade duration="{{ {dissolve_in}+{dissolve_offset} }}" />
				</Head>
				<Tail>
					<Fade duration="{{ {dissolve_out}+{dissolve_offset} }}" />	
					<Edit mode="duration" time="{{ {edit_out}+{dissolve_offset}-{edit_in} }}" />
				</Tail>
			</{media_type}>
"""

cml_footer_code = """
		</Segment>
		
		<Segment>
			<Canvas align="head" adjust="body" duration="{$$BlackTailTimecode}" Background="black" layer="0" />                           
		</Segment>

	</Sequence>

</Composition>
"""


# Open and Save Files

In [1724]:
# Opens EDL and extracts its contents into a single string.
def open_edl(edl_path):
    with open(edl_path, 'r') as edl_file:
        edl_contents = edl_file.read()
    edl_extract(edl_contents)    

In [ ]:
# Sets working directory
def set_current_directory ():
	pass

In [ ]:
#  Saves CML and CSV files to current directory
def save_files(cml_assembled, csv_assembled, isci, current_directory):
	cml = f"{current_directory}{isci}.cml"
	csv = f"{current_directory}{isci}.csv"
	
	with open(cml, 'w') as file:
		file.write(cml_assembled)

	with open(csv, 'w') as file:
		file.write(csv_assembled)

# Process the EDL

### Parse the EDL

In [ ]:
### Parse the EDL
# Breaks EDL into a list of fields for each line
def edl_extract(edl_contents): 
    edl_lines = edl_contents.splitlines()
    edl_extracted = []

    for line in edl_lines:
        columns = line.split()
        edl_extracted.append(columns)    
    
    simplify_edl(edl_extracted)


### Simplify EDL
Removes the extra columns and data from the EDL.  It also removes any lines where the in/out point match on the same line and no real edit is being made.

In [ ]:
### Simplify EDL
# Removes the extra columns and data from the EDL.  It also removes any lines where the in/out point match on the same line and no real edit is being made.
def simplify_edl(edl):
	detect_dropframe(edl)  #executes this function to determine dropframe is t/f and sets up some timecode
	simplified_edl = []

	for line in range(2, len(edl)):

		if edl[line][0] != "*" and edl[line][-1] != edl[line][-2]:  #if this doesn't have duplicate timecode for edits
			edits = {}
			dissolve_length = 0

			if edl[line][3] == "D":
				dissolve_length = int(edl[line][4])

			edits = {
				"media_type": edl[line][2][0],
				"edit_type": edl[line][3],
				"dissolve_length": dissolve_length,
				"edit_in" : edl[line][-2],
				"edit_out" : edl[line][-1]
				}

			simplified_edl.append(edits)  #  adds edits to the simplified EDL


		elif edl[line][0] == "*":  # lines with clip note
			# pprint(edl)
			edits = {
				"media_type" : edl[line][0],
				"media_file": edl[line][1]
			}

			simplified_edl.append(edits)  #  adds edl  to the simplified EDL 
	
	edl_split_video_audio(simplified_edl)	

### Split Video and Audio Edits

In [ ]:
### Split Video and Audio Edits
#  Looks for relevant edit information:  Drop/Non-Drop Frame, Title Name, Edits

def edl_split_video_audio(edl):
    # Clears the edit list for parsing
    global video_edits
    global audio_edits

    # pprint(edl)

    edl_video = []
    edl_audio = []

    for line in range(0, len(edl)):
        #  Splits audio and video edits into seperate EDLs
        if edl[line]["media_type"] == "V":
            edl_video.append(edl[line])
            if line+1 < len(edl) and edl[line+1]["media_type"] == "*":
                record =  edl[line+1]
                file = record["media_file"].replace(".nbtitle", "").replace(".NBTITLE", "").strip()  # removes the NB title extention if it was copied into the clip note in the EDL.  It'll be added back later.
                file += ".nbtitle"  # adds NB extension
                record["media_file"] = file  # updates the video media file with the proper NB extention
                edl_video.append(record)                

        if edl[line]["media_type"] == "A":
            edl_audio.append(edl[line])
            if line+1 < len(edl) and edl[line+1]["media_type"] == "*":
                edl_audio.append(edl[line+1])

    
    # Parses the edits
    if edl_video:
        video_edits = parse_edits(edl_video)
        

    if edl_audio:
        audio_edits = parse_edits(edl_audio)



### Parse Edits
Breaks out each edit out of the EDL.  It seperates the dissolves as their own clip.

In [ ]:
### Parse Edits
# Breaks out each edit out of the EDL.  It seperates the dissolves as their own clip.

def parse_edits(edl):
    edit_list = []

    for line in range(0, len(edl)):  
        edits = {} 

        #  Audio Edits
        if line+1 < len(edl) and edl[line]["media_type"] == "A":
            #  Audio - Cut In - Cut Out 
                # (only option - no audio dissolves done in vantage  - any edits needs to be done on the media file before exporting to vantage)
            edits = {
                "type": "audio_edit",
                "dissolve_in" : 0,
                "dissolve_out" : 0,
                "edit_in" : edl[line]["edit_in"],
                "edit_out" : edl[line]["edit_out"],
                "media_file" : "vo\\" + edl[line+1]["media_file"] + ".wav"
            }
            edit_list.append(edits)

                

        # #  Video Edits
        if (edl[line]["media_type"] == "V"):

            #  DISSOLVE IN
            if (line+2 < len(edl) and
                edl[line]["edit_type"] == "D" and
                edl[line+1]["media_type"] != "*" and
                edl[line+2]["media_type"] == "*"           
                ): 

                edits = {
                    "type" : "dissolve_in",
                    "dissolve_in" : edl[line]["dissolve_length"],
                    "dissolve_out" : 0,
                    "edit_in" : edl[line]["edit_in"],
                    "edit_out" : edl[line]["edit_out"],
                    "media_file" : edl[line+2]["media_file"]            
                }

                edit_list.append(edits)

            #  DISSOLVE OUT
            if (line+1 < len(edl) and
                edl[line]["edit_type"] == "D" and 
                edl[line+1]["media_type"] == "*"
                ): 
                     
                edits = {
                    "type" : "dissolve_out",
                    "dissolve_in" : 0,
                    "dissolve_out" : int(edl[line]["dissolve_length"]),
                    "edit_in" : edl[line]["edit_in"],
                    "edit_out" : edl[line]["edit_out"],
                    "media_file" : edl[line+1]["media_file"]            
                }
                  
                edit_list.append(edits)

            #  CUT IN CUT OUT
            if (line+1 < len(edl) and
                edl[line]["edit_type"] == "C" and 
                edl[line+1]["media_type"] == "*"
                ): 
                     
                edits = {
                    "type" : "cut_in_cut_out",
                    "dissolve_in" : 0,
                    "dissolve_out" : 0,
                    "edit_in" : edl[line]["edit_in"],
                    "edit_out" : edl[line]["edit_out"],
                    "media_file" : edl[line+1]["media_file"]            
                }
                  
                edit_list.append(edits)       

    consolodated_edits = consolodate_edits(edit_list)

    
    return consolodated_edits

### Timecode Tools

In [ ]:
### Timecode Tools

# Checks if EDL is dropframe and sets timecode formatting (; vs :)
def detect_dropframe(edl):
    global drop_frame
    global framerate
    global edit_hour_mark
    global dissolve_offset

     
    if edl[1][0] == "FCM:":
        if edl[1][1] == "DROP":
            drop_frame = True
            edit_hour_mark = f"01:00:00;00@{framerate}"
            dissolve_offset = f"00:00:00;01@{framerate}"

        else:
            drop_frame = False
            edit_hour_mark = f"01:00:00:00@{framerate}"
            dissolve_offset = f"00:00:00:01@{framerate}"


In [1732]:
# Convert timecode to dropframe
def format_timecode(timecode, framerate):
	global drop_frame

	if drop_frame == True:
		timecode=str(timecode.replace(":", ";").replace(";", ":", 2)) + "@" + str(framerate)
	
	else:
		timecode=str(timecode) + "@" + str(framerate)
	
	return timecode

In [1733]:
# Convert dissolves to timecode
def dissolves_timecode(dissolve, framerate):

	seconds = int(dissolve/round(framerate))
	frames = int(dissolve - (seconds*round(framerate)))

	if seconds < 10:
		seconds = f"0{seconds}"
	
	if frames < 10:
		frames = f"0{frames}"

	timecode = f"00:00:{seconds}:{frames}"

	return format_timecode(timecode, framerate)
	

###  Consolodate Edits
This adds the dissolves back into the the beginning/ending of applicable cuts-only edits.

In [ ]:
###  Consolodate Edits

# This adds the dissolves back into the the beginning/ending of applicable cuts-only edits.

def consolodate_edits(edl):
	new_edit_list = []
	
	for line in range(0, len(edl)):
		if edl[line]["type"] == "audio_edit":
			new_edits = {
                    "dissolve_in" : edl[line]["dissolve_in"],
                    "dissolve_out" : edl[line]["dissolve_out"],
                    "edit_in" : edl[line]["edit_in"],
                    "edit_out" : edl[line]["edit_out"],
                    "media_file" : edl[line]["media_file"],
					"media_type" : "Audio"            
                }
                  
			new_edit_list.append(new_edits)


		elif edl[line]["type"] == "cut_in_cut_out":
			new_in = edl[line]["edit_in"]
			new_out = edl[line]["edit_out"]
			new_dissolve_in = 0
			new_dissolve_out = 0
			new_edits = {}

			#  Dissolve In - Dissolve Out
			for check in range(0, len(edl)):
				if (edl[check]["type"] == "dissolve_in" and
					edl[line]["edit_in"] == edl[check]["edit_out"] and
					edl[line]["media_file"] == edl[check]["media_file"]
					):

					new_in = edl[check]["edit_in"]
					new_dissolve_in = edl[check]["dissolve_in"]


				if (edl[check]["type"] == "dissolve_out" and
					edl[line]["edit_out"] == edl[check]["edit_in"] and
					edl[line]["media_file"] == edl[check]["media_file"]
					):

					new_out = edl[check]["edit_out"]
					new_dissolve_out = edl[check]["dissolve_out"]

			new_edits = {
                    "dissolve_in" : new_dissolve_in,
                    "dissolve_out" : new_dissolve_out,
                    "edit_in" : new_in,
                    "edit_out" : new_out,
                    "media_file" : edl[line]["media_file"],
					"media_type" : "Video"           
                }
                  
			new_edit_list.append(new_edits)
	
	# Extract the media files from EDL (either new blue titles or audio files)
	edit_files = parse_media_files(new_edit_list)

	return new_edit_list


### Extract Media Files
Extract a list of media files from the EDL

In [ ]:
### Extract Media Files

# Extract the media files from EDL (either new blue titles or audio files)
def parse_media_files(edl):
    global media_files

    for media in edl:
        file = media["media_file"]

        if file not in media_files:
            media_files.append(file)


# Create Final Files 
Creates the files needed for Vantage (CML & CSV)

### Create CML File

In [ ]:
### Create CML File

def cml_sources(media_files, cml_media_code):    
    sources_code = ""

    for line in range(len(media_files)):
        source_number = line + 1
        media_file = media_files[line]

        sources_code += cml_media_code.format(
            source_number=source_number,
            media_file=media_file
        )    
    
    return sources_code

In [1737]:
def cml_edits(media_files, edits):
	global framerate
	global dissolve_offset
	global edit_hour_mark
	edit_code = ""

	if edits:
		for line in range(len(edits)):
			source_number = ""		
			media_file = edits[line]['media_file']

			#  Finds the media source number for this edit
			for index, source in enumerate(media_files):
				if source == media_file:
					source_number = index+1
					break
				
			
			# sends the variables to the edit code
			edit_code += cml_edit_code.format(
				source_number=source_number,
				media_type = edits[line]["media_type"],
				edit_in = format_timecode(edits[line]['edit_in'],framerate),
				edit_out = format_timecode(edits[line]['edit_out'], framerate),
				dissolve_in = dissolves_timecode(edits[line]['dissolve_in'], framerate),
				dissolve_out = dissolves_timecode(edits[line]['dissolve_out'], framerate),
				dissolve_offset = dissolve_offset,
				edit_hour_mark = edit_hour_mark
			)

	return edit_code

In [1738]:
def build_cml(media_files, video_edits, audio_edits):
	global cml_media_code
	global cml_slate_code
	global cml_header_code
	global cml_footer_code

	header = cml_header_code
	sources = cml_sources(media_files, cml_media_code)
	slate = cml_slate_code
	video_edits = cml_edits(media_files, video_edits)
	audio_edits = cml_edits(media_files, audio_edits)
	footer = cml_footer_code

	# Assembles CML code elements into a single file.
	cml_assembled = header + sources + slate + video_edits + audio_edits + footer
	
	return cml_assembled


### Create CSV File

In [ ]:
### Create CSV File

def build_csv(media_files):
	csv_assembled = ""

	for item in media_files:
		if item.endswith('nbtitle'):
			csv_assembled += f"{item}, TFN, URL, PROMO\n"
			csv_assembled += f"{item}, {{$$Workflow_TFN}}, {{$$Workflow_URL}}, {{$$Workflow_Promo}}\n" 

	return csv_assembled
	

# Execution 

In [ ]:
### Execution
open_edl(edl_path)

cml_assembled = build_cml(media_files, video_edits, audio_edits)
csv_assembled = build_csv(media_files)

save_files(cml_assembled, csv_assembled, isci, current_directory)